# xeus https://github.com/QuantStack/xeus

## A C++ implementation of the Jupyter Kernel Protocol

Xeus is a C++ implementation of the Jupyter Kernel Protocol

- BSD Licensed

xeus enables custom kernel authors to implement Jupyter kernels more easily. It takes the burden of implementing the Jupyter Kernel protocol so developers can focus on implementing the interpreter part of the Kernel.

**Installation:**

```bash
conda xeus-cling -c QuantStack -c conda-forge
```

## Output and error streams

`std::cout` and `std::cerr` are redirected to the notebook frontend.

In [ ]:
#include <iostream>

std::cout << "some output" << std::endl;

In [ ]:
std::cerr << "some error" << std::endl;

In [ ]:
#include <stdexcept>

In [ ]:
throw std::runtime_error("BAAAD");

Omitting the `;` in the last statement of a cell results in an output being printed

In [ ]:
int i = 4

In [ ]:
int j = 5;

In [ ]:
j

# Interpreting the C++ programming language

`cling` has a broad support of the features of C++. You can define functions, classes, templates, etc ...

## Functions

In [ ]:
double sqr(double a)
{
    return a * a;
}

In [ ]:
double a = 2.5;
double asqr = sqr(a);
asqr

## Classes

In [ ]:
class Foo
{
public:

    virtual ~Foo() {}
    
    virtual void print(double value) const
    {
        std::cout << "Foo value = " << value << std::endl;
    }
};

In [ ]:
Foo bar;
bar.print(1.2);

## Polymorphism

In [ ]:
class Bar : public Foo
{
public:

    virtual ~Bar() {}
    
    virtual void print(double value) const
    {
        std::cout << "Bar value = " << 2 * value << std::endl;
    }
};

In [ ]:
Foo* bar2 = new Bar;
bar2->print(1.2);
delete bar2;

## Templates

In [ ]:
#include <typeinfo>

template <class T>
class FooT
{
public:
    
    explicit FooT(const T& t) : m_t(t) {}
    
    void print() const
    {
        std::cout << typeid(T).name() << " m_t = " << m_t << std::endl;
    }
    
private:
    
    T m_t;
};

template <>
class FooT<int>
{
public:
    
    explicit FooT(const int& t) : m_t(t) {}
    
    void print() const
    {
        std::cout << "m_t = " << m_t << std::endl;
    }
    
private:
    
    int m_t;
};

In [ ]:
FooT<double> foot1(1.2);
foot1.print();

In [ ]:
FooT<int> foot2(4);
foot2.print();

## C++11 / C++14 support

In [ ]:
class Foo11
{
public:
    
    Foo11() { std::cout << "Foo11 default constructor" << std::endl; }
    Foo11(const Foo11&) { std::cout << "Foo11 copy constructor" << std::endl; }
    Foo11(Foo11&&) { std::cout << "Foo11 move constructor" << std::endl; }
};

In [ ]:
Foo11 f1;
Foo11 f2(f1);
Foo11 f3(std::move(f1));

In [ ]:
#include <vector>

std::vector<int> v = { 1, 2, 3};
auto iter = ++v.begin();
v

In [ ]:
*iter

... and also lambda, universal references, `decltype`, etc ...

## Documentation and completion

In [ ]:
?std::vector

## Using the `display_data` mechanism

For a user-defined type `T`, implementing the function `xeus::xjson mime_bundle_repr(const T* im)` returning the json mime bundle representation for that type enables the rich rendering in the notebook.

### Image example

In [ ]:
#include <string>
#include <fstream>

#include "xtl/xbase64.hpp"
#include "xeus/xjson.hpp"

namespace im
{
    struct image
    {   
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    xeus::xjson mime_bundle_repr(const image& i)
    {
        auto bundle = xeus::xjson::object();
        bundle["image/png"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}

In [ ]:
im::image marie("images/marie.png");
marie

### Audio example

In [ ]:
#include <string>
#include <fstream>

#include "xtl/xbase64.hpp"
#include "xeus/xjson.hpp"

namespace au
{
    struct audio
    {   
        inline audio(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    xeus::xjson mime_bundle_repr(const audio& a)
    {
        auto bundle = xeus::xjson::object();
        bundle["text/html"] =
           std::string("<audio controls=\"controls\"><source src=\"data:audio/wav;base64,")
           + xtl::base64encode(a.m_buffer.str()) +
            "\" type=\"audio/wav\" /></audio>";
        return bundle;
    }
}

In [ ]:
au::audio drums("audio/audio.wav");
drums